In [1]:
import pandas as pd

In [3]:
tokens = pd.read_csv('tokens_with_coref.csv')

tokens['hd_shifts'] = tokens['hd_shifts'].fillna(0)
tokens = tokens.drop('anaph', 1).drop('Unnamed: 0', 1).rename(index=str, columns={"hd_shifts": "anaph"}).fillna('-')
tokens.head()

,doc_id,shift_o,length_o,token,lemma,gram,ID,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,anaph
0,1,0,2,Во,во,Sp-a,1-0,-,-,-,-,-,-,-,-,-,-,0.0
1,1,3,5,время,время,Ncnsan,1-3,-,-,-,-,-,-,-,-,-,-,0.0
2,1,9,5,своих,свой,P---pga,1-9,1,407840,1070,0,9,5,своих,9,ref:def|str:refl|type:coref,-,0.0
3,1,15,8,прогулок,прогулка,Ncfpgn,1-15,-,-,-,-,-,-,-,-,-,-,0.0
4,1,24,1,в,в,Sp-l,1-24,-,-,-,-,-,-,-,-,-,-,0.0


In [4]:
sum(tokens['anaph'])

7386.0

In [5]:
import gensim
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format('./ruwikiruscorpora_0_300_20.bin', binary=True)

In [7]:
tokens['PoS'] = [g[0] for g in tokens['gram']]
tokens.groupby('PoS').first()

,doc_id,shift_o,length_o,token,lemma,gram,ID,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,anaph
PoS,,,,,,,,,,,,,,,,,,
",",1,82,1,",",",",",",1-82,-,-,-,-,-,-,-,-,-,-,0.0
-,1,336,1,-,-,-,1-336,-,-,-,-,-,-,-,-,-,-,0.0
A,1,69,8,одинокую,одинокий,Afpfsaf,1-69,-,-,-,-,-,-,-,-,-,-,0.0
C,1,270,1,И,и,C,1-270,-,-,-,-,-,-,-,-,-,-,0.0
I,1,3810,8,Хо-хо-хо,хо-хо-хо,I,1-3810,-,-,-,-,-,-,-,-,-,-,0.0
M,1,954,10,нескольких,несколько,Mc--g,1-954,-,-,-,-,-,-,-,-,-,-,0.0
N,1,3,5,время,время,Ncnsan,1-3,-,-,-,-,-,-,-,-,-,-,0.0
P,1,9,5,своих,свой,P---pga,1-9,1,407840,1070,0,9,5,своих,9,ref:def|str:refl|type:coref,-,0.0
Q,1,128,2,не,не,Q,1-128,-,-,-,-,-,-,-,-,-,-,0.0


In [9]:
tokens['vec'] = ['']*len(tokens)
pos_tags = {'V':'_VERB','A':'_ADJ','N':'_NOUN','R':'_ADV','Q':'_PART','P':'_PRON','I':'_INTJ', 'C':'_CCONJ', 'M':'ADV', 'S':'', ',':'', '-':''}
#c = 0
tokens['vec'] = tokens['vec'].astype('object')
for token_id, token in tqdm(tokens.iterrows()):
    #c += 1
    tag = token['lemma']+pos_tags[token['PoS']]
    if tag in model.vocab.keys():
        #print(model[tag])
        tokens.set_value(token_id, 'vec', model[tag])
    #if c == 10:
    #    break

In [10]:
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity

def mean_vec(vectors, count):
    res = [vectors[str(c)] for c in range(count)]
    print(np.mean( np.array(res), axis=0 ), count)
    return np.mean( np.array(res), axis=0 )
        

not_found = []
anaphora = pd.DataFrame({"target":[], "t_gend":[], "t_count":[], 't_vec-3':[], 't_vec-5':[], 
                         't_vec+3':[], 't_vec+5':[], 'discourse':[],
                         "cand_vec":[],"c_gend":[], "c_count":[],'c_pow':[], 'c_vec-3':[], 'c_vec-5':[], 
                         'c_vec+3':[], 'c_vec+5':[], 
                         'dif_vec-3':[], 'dif_vec-5':[], 'dif_vec+3':[], 'dif_vec+5':[],
                         'dist':[], 'dif_cand_disc':[], 'answ':[]})

In [28]:
def add_anaphora(i, row, anaphora):
    print(i, row)
    shift = row['shift']; target_link = row['link']; target = row['lemma']
                
    vec_count = 0; vec_after = {}
    while vec_count <= 5:
        if tokens.iloc[i+vec_count+1]['lemma'] not in punctuation:
            print(tokens.iloc[i+vec_count+1]['vec'][:5], 'vector')
            vec_after[str(vec_count)] = tokens.iloc[i+vec_count+1]['vec']
            vec_count += 1

    vec_count = 0; vec_before = {}
    while vec_count <= 30:
        if tokens.iloc[i-vec_count-1]['lemma'] not in punctuation:
            vec_before[str(vec_count)] = tokens.iloc[i-vec_count-1]['vec']
            vec_count += 1

    discource =  mean_vec(vec_before, 30)
    t_vec_minus3 = mean_vec(vec_before, 3); t_vec_minus5 = mean_vec(vec_before, 5)
    t_vec_plus3 = mean_vec(vec_after, 3); t_vec_plus5 = mean_vec(vec_after, 5)
    print(t_vec_minus3)

    if len(row['gram']) > 4:
        target_gend = row['gram'][4]; target_count = row['gram'][3]
    else:
        target_gend = '-'; target_count = '-'
    number_added = 0; found_answ = False

    while number_added <= 20:
        counter_words += 1; current_posit = i-counter_words-1
        if tokens.iloc[current_posit]['lemma'] not in punctuation:
            candidate_vec = tokens.iloc[current_posit]['vec']

            vec_count = 0; vec_before = {}
            while vec_count <= 5:
                if tokens.iloc[current_posit-vec_count-1]['lemma'] not in punctuation:
                    vec_before[str(vec_count)] = tokens.iloc[current_posit-vec_count-1]['vec']
                    vec_count += 1
            vec_count = 0; vec_after = {}
            while vec_count <= 5:
                if tokens.iloc[current_posit+vec_count+1]['lemma'] not in punctuation:
                    vec_after[str(vec_count)] = tokens.iloc[current_posit+vec_count+1]['vec']
                    vec_count += 1

            c_vec_minus3 = mean_vec(vec_before, 3); c_vec_minus5 = mean_vec(vec_before, 5)
            c_vec_plus3 = mean_vec(vec_after, 3); c_vec_plus5 = mean_vec(vec_after, 5)

            dif_vec_plus3 = cosine_similarity(c_vec_plus3, t_vec_plus3)
            dif_vec_plus5 = cosine_similarity(c_vec_plus5, t_vec_plus5)
            dif_vec_minus3 = cosine_similarity(c_vec_minus3, t_vec_minus3)
            dif_vec_minus5 = cosine_similarity(c_vec_minus5, t_vec_minus5)
            dif_cand_disc = cosine_similarity(discource, candidate_vec)

            if len(tokens.iloc[i-counter_words-1]['gram']) > 4:
                cand_gend = tokens.iloc[i-counter_words-1]['gram'][4]
                cand_count = tokens.iloc[i-counter_words-1]['gram'][3]
            else:
                cand_gend = '-'; cand_count = '-'
            cand_pow = tokens.iloc[i-counter_words-1]['gram'][0]
            dist = counter_words + 1
            number_added += 1
            print(number_added)
            if tokens.iloc[i-counter_words-1]['group_id'] == target_link:
                answ = 1
                found_answ = True
            else:
                answ = 0
            anaphora.loc[len(anaphora)+1] = [answ, c_count, c_gend, c_pow, c_vec_plus3, c_vec_plus5, c_vec_minus3,
                                               c_vec_minus5, candidate_vec, dif_cand_disc, dif_vec_plus3, dif_vec_plus5,
                                               dif_vec_minus3, dif_vec_minus5, discource, dist, t_count, t_gend,
                                               t_vec_plus3, t_vec_plus5, t_vec_minus3, t_vec_minus5, target]

    if not found_answ:
        #print('WARNING: answ_not_found for ', target, shift, i)
        not_found.append([target, shift, i])
add_anaphora(68, tokens.iloc[68], anaphora)

68 doc_id                                  1
shift_o                               371
length_o                                4
token                                ними
lemma                                 они
gram                              P-3-pin
ID                                  1-371
variant                                 1
group_id                           407850
chain_id                             1054
link                               407975
shift                                 371
length                                  4
content                              ними
tk_shifts                             371
attributes    ref:def|str:pron|type:anaph
head                                    -
anaph                                   1
PoS                                     P
vec                                      
Name: 68, dtype: object
 vector


KeyboardInterrupt: 

In [22]:
vec_after

{'0': ''}

In [19]:
for i, row in tqdm(tokens.iterrows()):
    counter_words = 0; i = int(i)
    if row['anaph'] == 1 and row['link'] != 0:
        add_anaphora(i, row, anaphora)
anaphora.head(40)

KeyboardInterrupt: 

In [16]:
anaphora


,answ,c_count,c_gend,c_pow,c_vec+3,c_vec+5,c_vec-3,c_vec-5,cand_vec,dif_cand_disc,...,dif_vec-5,discourse,dist,t_count,t_gend,t_vec+3,t_vec+5,t_vec-3,t_vec-5,target
